In [7]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.9 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/google')

Mounted at /content/google


In [16]:
# !/usr/bin/env python3
# -*- coding: utf-8 -*-

from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader
import torch
torch.manual_seed(0)
import numpy as np
np.random.seed(0)
import torch
from torch.optim import Adam
from tqdm import tqdm
from torch import nn
from torch.utils.data import Dataset
import numpy as np
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import string

class TweetDataset(Dataset):
    def __init__(self, df, tokenizer):
        texts = df.text.values.tolist()
        texts = [self._preprocess(text) for text in texts]
        self._print_random_samples(texts)
        self.texts = [texts]
        if 'target' in df:
            classes = df.target.values.tolist()
            self.labels = classes

    def _print_random_samples(self, texts):
        np.random.seed(42)
        random_entries = np.random.randint(0, len(texts), 5)

        for i in random_entries:
            print(f"Entry {i}: {texts[i]}")


    def _preprocess(self, text):
        text = self._remove_amp(text)
        text = self._remove_links(text)
        text = self._remove_hashes(text)
        text = self._remove_retweets(text)
        text = self._remove_mentions(text)
        text = self._remove_multiple_spaces(text)

        # text = self._lowercase(text)
        text = self._remove_punctuation(text)
        # text = self._remove_numbers(text)

        text_tokens = self._tokenize(text)
        text_tokens = self._stopword_filtering(text_tokens)
        # text_tokens = self._stemming(text_tokens)
        text = self._stitch_text_tokens_together(text_tokens)

        return text.strip()

    def _remove_amp(self, text):
        return text.replace("&amp;", " ")

    def _remove_mentions(self, text):
        return re.sub(r'(@.*?)[\s]', ' ', text)

    def _remove_multiple_spaces(self, text):
        return re.sub(r'\s+', ' ', text)

    def _remove_retweets(self, text):
        return re.sub(r'^RT[\s]+', ' ', text)

    def _remove_links(self, text):
        return re.sub(r'https?:\/\/[^\s\n\r]+', ' ', text)

    def _remove_hashes(self, text):
        return re.sub(r'#', ' ', text)

    def _stitch_text_tokens_together(self, text_tokens):
        return " ".join(text_tokens)

    def _tokenize(self, text):
        return nltk.word_tokenize(text, language="english")

    def _stopword_filtering(self, text_tokens):
        stop_words = nltk.corpus.stopwords.words('english')

        return [token for token in text_tokens if token not in stop_words]

    def _stemming(self, text_tokens):
        porter = nltk.stem.porter.PorterStemmer()
        return [porter.stem(token) for token in text_tokens]

    def _remove_numbers(self, text):
        return re.sub(r'\d+', ' ', text)

    def _lowercase(self, text):
        return text.lower()

    def _remove_punctuation(self, text):
        return ''.join(character for character in text if character not in string.punctuation)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]

        label = -1
        if hasattr(self, 'labels'):
            label = self.labels[idx]
        return text, label

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [65]:
import csv
import pandas as pd
test_project = pd.read_csv('./tweets.csv',  on_bad_lines='skip',delimiter=",",
              header=0, encoding='ISO-8859-1')
test_project = test_project[test_project['language']=='en'][['date','content']]
print(test_project.head(5))
test_project.rename(columns={'content':'text'}, inplace=True)
test_project['date'] = pd.to_datetime(test_project['date'])
print(len(test_project))

<ipython-input-65-fd0f85fff917>:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  test_project = pd.read_csv('./tweets.csv',  on_bad_lines='skip',delimiter=",",


                        date  \
1  2023-02-21 03:29:07+00:00   
2  2023-02-21 03:29:04+00:00   
3  2023-02-21 03:28:06+00:00   
5  2023-02-21 03:27:27+00:00   
6  2023-02-21 03:27:11+00:00   

                                             content  
1  New search &amp; rescue work is in progress in...  
2  Can't imagine those who still haven't recovere...  
3  its a highkey sign for all of us to ponder ove...  
5  See how strong was the #Earthquake of Feb 20, ...  
6  More difficult news today on top of struggles ...  
189626


In [ ]:
test_project.head(10)

In [66]:
import datetime
three_days = datetime.date(2023, 2, 8)
one_week = datetime.date(2023, 2, 12)
test_3d = test_project[test_project['date'].dt.date<=three_days]
test_1w = test_project[test_project['date'].dt.date<=one_week][test_project['date'].dt.date>three_days]
test_after_1w = test_project[test_project['date'].dt.date>one_week]
print(len(test_3d), len(test_1w), len(test_after_1w))

130125 49307 10194


<ipython-input-66-563cf3a3b788>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test_1w = test_project[test_project['date'].dt.date<=one_week][test_project['date'].dt.date>three_days]


In [62]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("TweebankNLP/bertweet-tb2_wnut17-ner")

model = AutoModelForTokenClassification.from_pretrained("TweebankNLP/bertweet-tb2_wnut17-ner")



def word_extraction(words, labels):
  entities = []
  current_entity = None

  for word, label in zip(words, labels):
      if label.startswith("B-"):
          # start of a new entity
          if current_entity is not None:
              entities.append(current_entity)
          current_entity = (word, label[2:])
      elif label.startswith("I-"):
          # continuation of current entity
          if current_entity is None:
              current_entity = (word, label[2:])
          else:
              current_entity = (current_entity[0] + " " + word, current_entity[1])
      else:
          # end of current entity
          if current_entity is not None:
              entities.append(current_entity)
              current_entity = None

  # add last entity if exists
  if current_entity is not None:
      entities.append(current_entity)

  # print(entities)
  return entities

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [76]:
##### Prediction
import torch
from torch.optim import Adam
from tqdm import tqdm
def get_text_predictions(model, loader):
    device = torch.device('cuda')
    model = model.to(device)
    ner_results = {}
    with torch.no_grad():
        model.eval()
        for data_input, _ in tqdm(loader):
          for s in data_input:
            tweet = s[0]
            # print('dddd', s[0])
            # input_ids = data_input['input_ids'].squeeze(1).to(device)
            # tweet = data_input
            inputs = tokenizer.encode(tweet, add_special_tokens=False, return_tensors="pt", truncation=True)
            # print('iiiiiii', inputs)
            inputs = inputs.squeeze(1).to(device)
            # output = model(inputs).logits
            logits = model(inputs).logits
            # print('llll',logits)
            predicted_token_class_ids = logits.argmax(-1)
            predicted_tokens_classes = [model.config.id2label[t.item()] for t in predicted_token_class_ids[0]]
            # print('pppp',predicted_tokens_classes)
            entities = word_extraction(tweet.split(), predicted_tokens_classes)
            for t in entities:
              if t[0] not in ner_results:
                ner_results[t[0]] = 1
              else:
                ner_results[t[0]] += 1
    ner_results = {k:v for k,v in ner_results.items() if v>=5}
    sorted_ner = dict(sorted(ner_results.items(), reverse=True, key= lambda items: items[1]))
    print('rrrr', sorted_ner)
    return sorted_ner

In [ ]:
time_list = [test_3d, test_1w, test_after_1w]
with open('./ner_results.csv', 'w') as f:
    f.write('time \n')
    for i in range(len(time_list)):
        if i == 0:
            t = '0 to 3 days'
        elif i == 1:
            t = '4 to 7 days'
        else:
            t = '8 to 16 days'
        test_dataloader = DataLoader(TweetDataset(time_list[i][:1000], tokenizer),
                                     batch_size=8,
                                     shuffle=False,
                                     num_workers=0)
    
        r = get_text_predictions(model, test_dataloader)
        

Entry 102: Incomprehensible levels devastation despair suffering Aleppo Itâs 4Â°c tonight people still rubble still digging bare hands SaveNorthSyria earthquake TurkeySyriaEarthquake Ø§ÙÙØ°ÙØ§Ø§ÙØ´Ù Ø§ÙØ§ÙØ³ÙØ±Ù
Entry 435: The Saudi government opened account residents transfer donations Syria Turkey Within couple hours 200000 people donated total 75 million riyals â 20 million dollars
Entry 860: ð Earthquake deprem M34 occurred 28 km E KahramanmaraÅ Turkey 8 min ago local time 003554 More info ð± ð ð¥
Entry 270: nonot appstoreFailure turkey problem
Entry 106: Beware Turkey Syeria Earthquake Fake Donation Sites Please dont donate unknown organization There least 144 fake agencies various networks collecting donations None legitimate organizations Contact Turkey Syerias local embassy


100%|██████████| 1/1 [00:12<00:00, 12.26s/it]


rrrr {'Turkey': 180, 'Syria': 162, 'TurkeySyriaEarthquake': 37, 'earthquake': 32, 'Twitter': 23, 'Turkey Syria': 21, 'Turkish': 15, 'US': 14, 'turkey': 13, 'info': 13, 'twitter': 13, 'Earthuack2023': 13, 'Syrians': 11, 'TurkeyQuake': 10, 'Syrian': 10, 'HelpTurkey': 9, 'TurkeyEarthquake': 9, 'earthquake italyearthquake': 9, 'min ago More': 8, 'Earthquake': 7, 'Turkiye Syria': 6, 'This': 6, 'people': 6, 'earthquakeinturkey': 6, 'TURKIYE': 5, 'TÃ¼rkiye': 5, 'Ukraine': 5, 'aid': 5}
Entry 102: Heroes WhiteHelmets Syria Earthquake SyriaEarthquake
Entry 435: 2 Istanbul 166 hours quake sondakika Turkiye 29605 died incl 2 Indonesian 89 Palestinian Suat Bayram Australian 375 Syrian 7 Iraqi Syria 5279 died Death Toll 34884 Surpassed 34000 died Nearly 35000 depremden
Entry 860: WATCH Senior paramedic head disaster response Israels emergency medical service provides update minutes team rescued 12yearold boy 23yearold woman Turkey earthquakes aftermath TurkeySyriaEarthquake Via MDA
Entry 270: But id

  0%|          | 0/1 [00:00<?, ?it/s]